In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df = pd.read_csv('/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv')

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
print(df.RainTomorrow.isnull().any())
print(df.count().isnull().any().any())

In [ ]:
df.isna().any()

# Data Insight

I've noticed a min and max temp, lets create and avg temp column using these features.

In [ ]:
sns.set()
df.hist(sharex = False, sharey = False, xlabelsize = 4, ylabelsize = 4, figsize = (12, 8))
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
plt.title('Correlation', size=15)
sns.heatmap(df.corr(), annot=True)

In [ ]:
corr_matrix = df.corr()

corr_matrix['Rainfall'].sort_values(ascending=False)

In [ ]:
import plotly.express as px

In [ ]:
df_dropped_na = df.dropna()


y_today = df_dropped_na['RainToday']
y_tomorrow = df_dropped_na['RainTomorrow']
df_dropped_na = df_dropped_na.drop(columns=['RainToday','RainTomorrow'])
X_today = df_dropped_na
X_tomorrow = df_dropped_na

# RainTomorrow?

Some preprocessing pipelines created for efficiency. We will compare a Support Vector Classifier (SVC) and a
Random Forest Classifier.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

X_train_tomorrow, X_test_tomorrow, y_train_tomorrow, y_test_tomorrow = train_test_split(X_tomorrow, y_tomorrow, test_size=.2, random_state=42)

cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

cat_attributes = X_today.select_dtypes(include=['object']).columns

num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])
num_attributes = df.select_dtypes(include=['float64']).columns.values

preprocessor = ColumnTransformer([
    ('cat', cat_transformer, cat_attributes),
    ('num', num_transformer, num_attributes)
])

clf_svc_tomorrow = Pipeline(steps=[('preprocessor', preprocessor),
               ('classifier', SVC(random_state=42))])

clf_forest_tomorrow = Pipeline(steps=[('preprocessor', preprocessor),
               ('classifier', RandomForestClassifier(random_state=42))])

In [ ]:
fig = plt.figure(figsize=(20,12),facecolor='white')

fig.suptitle('Univariate Distributions', weight='bold', fontsize='xx-large', fontweight='bold')

gs = fig.add_gridspec(2,2)

ax = [_ for i in range(4)]

ax[0] = fig.add_subplot(gs[0,0])
ax[1] = fig.add_subplot(gs[0,1])
ax[2] = fig.add_subplot(gs[1,0])
ax[3] = fig.add_subplot(gs[1,1])

sns.histplot(x='Temp3pm', data=X_train_tomorrow, ax=ax[0], shrink=0.5, bins=50, color='lightcoral')
sns.histplot(x='Humidity3pm', data=X_train_tomorrow, ax=ax[3], bins=20, color='darkred')
sns.boxplot(x='Cloud3pm', data=X_train_tomorrow, ax=ax[1], color='gold')
sns.violinplot(x='Sunshine', data=X_train_tomorrow, ax=ax[2], color='greenyellow')

for i in range(4):
    
    ax[i].set_yticklabels('')
    ax[i].set_ylabel('')
    ax[i].tick_params('y', length=0)
    
    for direction in ['top', 'left', 'right']:
        
        ax[i].spines[direction].set_visible(False)

In [ ]:
fig = plt.figure(figsize=(20,12), facecolor='white')

plt.suptitle('Relationship b/t two continuous variables', fontsize='xx-large', weight='bold')

gs=fig.add_gridspec(2,2)

ax = [None for _ in range(3)]

ax[0] = fig.add_subplot(gs[1,:])
ax[1] = fig.add_subplot(gs[0,0])
ax[2] = fig.add_subplot(gs[0,1])

sns.lineplot(data=X_train_tomorrow, x='Sunshine', y='Rainfall', color='gold', ax=ax[0])
sns.scatterplot(data=X_train_tomorrow, x='Cloud3pm', y='Rainfall', color='sienna', ax=ax[1], alpha=.1)
sns.scatterplot(data=X_train_tomorrow, x='Humidity3pm', y='Rainfall', color='blue', marker='x', alpha=.1,ax=ax[2])

for i in range(3):
    
    ax[i].set_yticklabels('')
    ax[i].set_ylabel('')
    ax[i].tick_params('y', length=0)
    
    for direction in ['top','left','right']:
        
        ax[i].spines[direction].set_visible(False)
    pass

fig.legend()

In [ ]:
fig = plt.figure(figsize=(20,12),facecolor='white')

plt.suptitle('Relationship b/t continuos variables and one categorical variable',
            fontsize='xx-large', fontweight='bold')

ax = [_ for i in range(3)]

ax[0] = fig.add_subplot(gs[0,0])
ax[1] = fig.add_subplot(gs[0,1])
ax[2] = fig.add_subplot(gs[1,:])

for i in range(3):
    ax[i].set_yticklabels([])
    ax[i].tick_params(axis='y', length=0)
    
    for direction in ['top', 'right', 'left']:
        ax[i].spines[direction].set_visible(False)
        
sns.lineplot(data=df, x='Sunshine', y='Rainfall', hue='RainTomorrow', ax=ax[0])
sns.scatterplot(data=df, x='Cloud3pm', y='Rainfall', hue='RainTomorrow', alpha=.1, ax=ax[1])
sns.scatterplot(data=df, x='Humidity3pm', y='Rainfall', hue='RainTomorrow', alpha=.1, ax=ax[2])

In [ ]:
forest_accuracy = clf_forest_tomorrow.fit(X_train_tomorrow, y_train_tomorrow.values.ravel()).score(X_train_tomorrow, y_train_tomorrow.values.ravel())
#svc_accuracy = clf_svc_tomorrow.fit(X_train_tomorrow, y_train_tomorrow.values.ravel()).score(X_train_tomorrow, y_train_tomorrow.values.ravel())

print('forest_accuracy: ', forest_accuracy, '\n')
#print('svc_accuracy: ', svc_accuracy, '\n')


Lets get some insight on how well our classifier is doing.

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix


# clf_svc_tomorrow_predict = cross_val_predict(clf_svc_tomorrow, X_train_tomorrow, y_train_tomorrow.values.ravel(), cv=3, method='decision_function')
clf_forest_tomorrow_predict = cross_val_predict(clf_forest_tomorrow, X_train_tomorrow, y_train_tomorrow.values.ravel(), cv=3)

clf_forest_tomorrow_score = cross_val_score(clf_forest_tomorrow, X_train_tomorrow, y_train_tomorrow.values.ravel(), cv=3)

print('cross_val_score: ', clf_forest_tomorrow_score)
print('cross_val_predict: ', clf_forest_tomorrow_predict)

In [ ]:
conf_mx = confusion_matrix(y_train_tomorrow.values.ravel(), clf_forest_tomorrow_predict)

In [ ]:
plt.matshow(conf_mx)
plt.show()

In [ ]:
y_train_tomorrow.head()

In [ ]:
precisions, recalls, thresholds = precision_recall_curve(y_train_tomorrow, clf_forest_tomorrow_predict, pos_label='Yes')


def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label='Precision')
    plt.plot(thresholds, recalls[:-1], "g-", label='Recall')
    plt.legend(loc="center right", fontsize=12)
    plt.xlabel("Threshold", fontsize=16)
    plt.grid(True)
    
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.show()

In [ ]:
plt.plot(recalls[:-1], precisions[:-1],'p-', linewidth=.05, label='precision')
plt.legend(loc='center right', fontsize=12)
plt.xlabel('recalls', fontsize=16)
plt.grid(True)
plt.show()

In [ ]:
# Need to fix this method

from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_train_tomorrow, clf_svc_tomorrow_predict, pos_label='Yes')

def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0,1], [0,1], 'k--')
    plt.xlabel('fpr')
    plt.ylabel('tpr')
    plt.grid(True)
    plt.legend(loc='center right', fontsize=12)
    plt.show()
    
plot_roc_curve(fpr, tpr, label='Yes')

Did I overfit this data? To what degree are my parameters being regularized? Need to check for variance and bias.

In [ ]:
# i gotta fix this and look over

import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate 3 plots: the test and training learning curve, the training
    samples vs fit times curve, the fit times vs score curve.

    Parameters
    ----------
    estimator : estimator instance
        An estimator instance implementing `fit` and `predict` methods which
        will be cloned for each validation.

    title : str
        Title for the chart.

    X : array-like of shape (n_samples, n_features)
        Training vector, where ``n_samples`` is the number of samples and
        ``n_features`` is the number of features.

    y : array-like of shape (n_samples) or (n_samples, n_features)
        Target relative to ``X`` for classification or regression;
        None for unsupervised learning.

    axes : array-like of shape (3,), default=None
        Axes to use for plotting the curves.

    ylim : tuple of shape (2,), default=None
        Defines minimum and maximum y-values plotted, e.g. (ymin, ymax).

    cv : int, cross-validation generator or an iterable, default=None
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:

          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, default=None
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like of shape (n_ticks,)
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the ``dtype`` is float, it is regarded
        as a fraction of the maximum size of the training set (that is
        determined by the selected validation method), i.e. it has to be within
        (0, 1]. Otherwise it is interpreted as absolute sizes of the training
        sets. Note that for classification the number of samples usually have
        to be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, 'o-')
    axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,
                         fit_times_mean + fit_times_std, alpha=0.1)
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
    axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1)
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt

In [ ]:
# lets just focus on our RainTomorrow classifier.
title = r"Learning Curves (SVM)"
#cv = ShuffleSplit(n_splits=10, test_size=.2, random_state)

fig, axes = plt.subplots(3, 2, figsize=(10, 15))
plot_learning_curve(clf_svc_tomorrow, title, X_train_tomorrow, y_train_tomorrow, cv=3)
